# **Laboratorio 13: 💨 Airflow 💨**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Cristopher Urbina H.
- Nombre de alumno 2: Joaquín Zamora O.

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/CrisU8/MDS7202-Primavera2024)

## Temas a tratar

- Construcción de pipelines productivos usando `Airflow`.


## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Reconocer los componentes pricipales de `Airflow` y su funcionamiento.
- Poner en práctica la construcción de pipelines de `Airflow`.
- Automatizar procesos típicos de un proyecto de ciencia de datos mediante `Airflow` y `Docker`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://i.gifer.com/SUFL.gif" width="400">
</p>

Nico, un estudiante del Magíster en Ciencia de Datos, se encuentra en la etapa final de sus estudios. Por un lado, está muy contento por haber llegado tan lejos, pero por otro, no puede evitar sentirse inquieto. Desde que ingresó a la universidad, una pregunta lo ha perseguido: ¿qué tan probable es que pueda ser seleccionado en los lugares donde envíe postulaciones para puestos de trabajo?

Esta duda lo mantiene en constante reflexión, especialmente porque sabe que el mercado laboral en Ciencia de Datos es competitivo y exige habilidades no solo técnicas, sino también estratégicas para destacar. Sin embargo, Nico actualmente está completamente enfocado en terminar su tesis de magíster y ha tenido que postergar cualquier preparación específica para enfrentar el desafío de las postulaciones laborales.

Al ver el avance y las habilidades que usted ha demostrado en el curso, Nico decidió proponerle un desafío que le permitirá disminuir la incertidumbre sobre su futuro laboral. Inspirado en sus conocimientos, él recolectó un conjunto de datos que contiene información sobre diversos factores que influyen en las decisiones de contratación de empresas al seleccionar entre sus postulantes. Este set de datos incluye los siguientes atributos:

- Age: Edad del candidato
- Gender: Genero del candidato. Male (0), Female (1).
- EducationLevel: Mayor nivel educacional alcanzado por el candidato. Licenciatura Tipo 1 (1), Licenciatura Tipo 2 (2), Maestría (3), PhD. (4).
- ExperienceYears: Años de experiencia profesional.
- PreviousCompanies: Numero de compañías donde el candidato ha trabajado anteriormente.
- DistanceFromCompany: Distancia en kilometros entre la residencia del candidato y la compañía donde postula.
- InterviewScore: Puntaje obtenido en la entrevista por el candidato entre 0 a 100.
- SkillScore: Puntaje obtenido en evaluación de habilidades técnicas por el candidato, entre 0 a 100.
- PersonalityScore: Puntaje obtenido en pruebas de personalidad del candidato, entre 0 a 100.
- RecruitmentStrategy: Estrategia del equipo de reclutamiento. Agresiva (1), Moderada (2), Conservadora (3).

Variable a predecir:
- HiringDecision: Resultado de la postulación. No contratado (0), Contratado (1).

Su objetivo será ayudar a Nico a desarrollar un modelo que le permita predecir, basado en estos factores, si un postulante será contratado o no. Esta herramienta no solo le dará a Nico mayor claridad sobre el impacto de ciertos atributos en la decisión final de contratación, sino que también le permitirá aplicar sus conocimientos de Ciencia de Datos para resolver una pregunta que a muchos estudiantes como él les inquieta.

Como estudiante del curso Laboratorio de Programación Científica para Ciencia de Datos, deberá demostrar sus capacidades para preprocesar, analizar y modelar datos, brindándole a Nico una solución robusta y bien fundamentada para su problemática.

`Nota:` El siguiente [enlace](https://www.kaggle.com/datasets/rabieelkharoua/predicting-hiring-decisions-in-recruitment-data/data) contiene el set de datos original.

# **1. Pipeline de Predicción Lineal** (30 Puntos)

<p align="center">
  <img src="https://c.tenor.com/WvHhQt2UpuAAAAAd/wolf-of-wall-street.gif" width="400">
</p>

En esta sección buscaremos desplegar un producto utilizando un modelo de clasificación `Random Forest` para determinar si una persona será contratada o no en un proceso de selección. Para ello, comenzaremos preparando un pipeline lineal mediante `Airflow`.

## **1.1 Preparando el Pipeline** (15 puntos)

**Primero, asegúrese de tener creada las carpetas `dags`, `plugins` y `logs`**.

Comenzamos preparando un archivo llamado `hiring_functions.py`, el cual guardará en la carpeta `dags` y debe contener lo siguiente:

1. (3 puntos) Una función llamada `create_folders()` que cree una carpeta, la cual utilice la fecha de ejecución como nombre. Adicionalmente, dentro de esta carpeta debe crear las siguientes subcarpetas:
  - raw
  - splits
  - models

  `Hint`: Puede hacer uso de kwargs para obtener la fecha de ejecución mediante el DAG. El siguiente [Enlace](https://airflow.apache.org/docs/apache-airflow/stable/templates-ref.html) le puede ser útil.
2. (3 puntos) Una función llamada `split_data()` que lea el archivo `data_1.csv` de la carepta `raw` y a partir de este, aplique un *hold out*, generando un dataset de entrenamiento y uno de prueba. Luego debe guardar estos nuevos conjuntos de datos en la carpeta `splits`. `Nota:` Utilice un 20% para el conjunto de prueba, mantenga la proporción original en la variable objetivo y fije una semilla.
3. (8 puntos) Cree una función llamada `preprocess_and_train()` que:
  - Lea los set de entrenamiento y prueba de la carpeta `splits`.
  - Cree y aplique un `Pipeline` con una etapa de preprocesamiento. Utilice `ColumnTransformers` para aplicar las transformaciones que estime convenientes. Puede apoyarse del archivo `data_1_report.html` para justificar cualquier paso del preprocesamiento.
  
  - Añada una etapa de entrenamiento utilizando el modelo `RandomForest`.
  
  Esta función **debe crear un archivo `joblib` (análogo a `pickle`) con el pipeline entrenado** en la carepta `models`, además debe **imprimir** el accuracy en el conjunto de prueba y el f1-score de la clase positiva (contratado).
3. (1 punto) Incorpore la función `gradio_interface` en su script, modificando la ruta de acceso a su modelo, de forma que pueda leerlo desde la carepta `models`. Puede realizar modificacioneds adicionales en caso de ser necesario.

`NOTA:` Se permite la creación de funciones auxiliares si lo estiman conveniente.

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score
import joblib
import gradio as gr


# 1. Función para crear carpetas
def create_folders(execution_date):
    base_folder = os.path.join(os.getcwd(), f"output_{execution_date}")
    subfolders = ["raw", "splits", "models"]
    os.makedirs(base_folder, exist_ok=True)
    for subfolder in subfolders:
        os.makedirs(os.path.join(base_folder, subfolder), exist_ok=True)
        print(f"Creating folders for {os.path.join(base_folder, subfolder)}")



# 2. Función para dividir los datos
def split_data(execution_date):
    # Define rutas basadas en la fecha de ejecución
    base_path = os.getcwd()
    input_path = os.path.join(base_path, f"output_{execution_date}/raw/data_1.csv")
    split_folder = os.path.join(base_path, f"output_{execution_date}/splits")

    # Verifica si el archivo de entrada existe
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"No se encontró el archivo: {input_path}")

    # Lee el archivo y divide los datos
    data = pd.read_csv(input_path)
    if "HiringDecision" not in data.columns:
        raise ValueError("La columna 'HiringDecision' no está en los datos.")

    X = data.drop(columns=["HiringDecision"])
    y = data["HiringDecision"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # Guarda los conjuntos divididos
    pd.concat([X_train, y_train], axis=1).to_csv(os.path.join(split_folder, "train.csv"), index=False)
    pd.concat([X_test, y_test], axis=1).to_csv(os.path.join(split_folder, "test.csv"), index=False)
    print(f"Archivos guardados en {split_folder}")



# 3. Función para preprocesar y entrenar modelo
def preprocess_and_train(execution_date):
    # Leer datasets
    base_path = os.getcwd()
    train_path = os.path.join(base_path, f"output_{execution_date}/splits/train.csv")
    test_path = os.path.join(base_path, f"output_{execution_date}/splits/test.csv")
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    X_train = train_data.drop(columns=["HiringDecision"])
    y_train = train_data["HiringDecision"]
    X_test = test_data.drop(columns=["HiringDecision"])
    y_test = test_data["HiringDecision"]

    # Preprocesamiento
    numeric_features = ["Age", "ExperienceYears", "DistanceFromCompany", "InterviewScore", "SkillScore",
                        "PersonalityScore"]
    categorical_features = ["Gender", "EducationLevel", "PreviousCompanies", "RecruitmentStrategy"]

    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
    categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    # Pipeline de modelo
    model = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(random_state=42)),
        ]
    )

    # Entrenar modelo
    model.fit(X_train, y_train)

    # Evaluación
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score (Positive Class): {f1:.4f}")

    # Guardar modelo entrenado
    model_path = f"output_{execution_date}/models/hiring_model.joblib"
    joblib.dump(model, model_path)


def predict(file,model_path):

    pipeline = joblib.load(model_path)
    input_data = pd.read_json(file)
    predictions = pipeline.predict(input_data)
    print(f'La prediccion es: {predictions}')
    labels = ["No contratado" if pred == 0 else "Contratado" for pred in predictions]

    return {'Predicción': labels[0]}


def gradio_interface(execution_date):

    model_path= f'output_{execution_date}/models/hiring_model.joblib'

    interface = gr.Interface(
        fn=lambda file: predict(file, model_path),
        inputs=gr.File(label="Sube un archivo JSON"),
        outputs="json",
        title="Hiring Decision Prediction",
        description="Sube un archivo JSON con las características de entrada para predecir si Nico será contratado o no."
    )
    interface.launch(share=True)


## **1.2 Creando Nuestro DAG** (15 puntos)

<p align="center">
  <img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExMzNjd3hxOWIzZjhwZDc5NnJwZzZodnNrbWI5cGtjY2VwZjI0eDdnOSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/Dh5q0sShxgp13DwrvG/giphy.webp" width="400">
</p>

Con las funciones del pipeline ya creadas, ahora vamos a proceder a crear un Directed Acyclic Graph (DAG). Para ello, se le pide lo siguiente:

- (10 puntos) Cree un segundo archivo llamado `dag_lineal.py` y guardelo en la carpeta dags. Este script debe seguir la siguiente estructura (Ver imagen de referencia):

    0. Inicialice un DAG con fecha de inicio el 1 de octubre de 2024, ejecución manual y **sin backfill**. Asigne un `dag_id` que pueda reconocer facilmente, como `hiring_lineal`, etc.
    1. Debe comenzar con un marcador de posición que indique el inicio del pipeline.
    2. Cree una carpeta correspondiente a la ejecución del pipeline y cree las subcarpetas `raw`, `splits` y `models` mediante la función `create_folders()`.
    3. Debe descargar el archivo `data_1.csv` del siguiente [enlace](https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv). Debe guardar el archivo en la carpeta raw de la ejecución correspondiente.`Hint:` Le puede ser útil el comando `curl -o <path de guardado> <enlace con los datos>`.
    4. Debe aplicar un hold out mediante la función `split_data()` de su archivo creado en la subsección anterior.
    5. Debe aplicar el preprocesamiento y el entrenamiento del modelo mediante la función `preprocess_and_train()`.
    6. Finalmente, debe montar una interfaz en gradio donde pueda cargar un archivo ``json``.


- (3 puntos) Cree un `DockerFile` para montar un contenedor que contenga Airflow. Adicionalmente, cree una carpeta llamada dags donde guardará el script.py creado anteriormente.

    `Nota:` Para la imagen, se recomienda utilizar python 3.10-slim. Adicionalmente, puede instalar `curl` mediante la siguiente linea de código: `RUN apt-get update && apt-get install -y curl`.

- Construya el contenedor en Docker y acceda a la aplicación web de Airflow mediante el siguiente [enlace](http://localhost:8080/). Inicie sesión, acceda al DAG creado y ejecute de forma manual su pipeline.

- (2 puntos) Acceda a la URL pública de Gradio e ingrese el archivo `nico_data.json` a su modelo. ¿Que predicción entregó el modelo para Nico? Adjunte imágenes de su resultado. `Hint:` Puede acceder a los `logs` para obtener los prints y la URL pública.

`Hint:` Recuerde que puede entregar `kwargs` a sus funciones, como por ejemplo la fecha de ejecución `ds`.

**Para esta sección, debe adjuntar todos los scripts creados junto a su notebook en la entrega, ya que serán ejecutados para validar el funcionamiento. Para justificar sus respuestas, adicionaslmente puede utilizar imágenes de apoyo, como screenshots.**

DAG de referencia:
<p align="center">
  <img src="https://drive.google.com/uc?id=1iwDgECZfFeWq1dl433tMa6_3CNF9cn1L" width="1200">
</p>




In [ ]:
from airflow import DAG
from airflow.operators.dummy import DummyOperator
from airflow.operators.python import PythonOperator
from airflow.operators.bash import BashOperator
from datetime import datetime
from hiring_functions import create_folders, split_data, preprocess_and_train, gradio_interface


# Inicialización del DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
}

# Definición del DAG
with DAG(
    dag_id="hiring_lineal",
    default_args=default_args,
    description="Pipeline para predicción de contratación",
    schedule_interval=None,  # Ejecución manual
    start_date=datetime(2024, 10, 1),
    catchup=False,  # Sin backfill
    params={}
) as dag:

    # 1. Marcador de inicio
    start = DummyOperator(task_id="start_pipeline")

    # 2. Crear carpetas
    create_folders_task = PythonOperator(
        task_id="create_folders",
        python_callable=create_folders,
        op_kwargs={"execution_date": "{{ ds }}"},
    )

    # 3. Descargar datos
    download_data_task = BashOperator(
        task_id='download_dataset',
        bash_command=(
            "curl -o /root/airflow/output_{{ ds }}/raw/data_1.csv "
            "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv"
        ),
    )

    # 4. Aplicar hold out
    split_data_task = PythonOperator(
        task_id="split_data",
        python_callable=split_data,
        provide_context=True,
        op_kwargs={"execution_date": "{{ ds }}"}
    )

    # 5. Preprocesar y entrenar
    preprocess_and_train_task = PythonOperator(
        task_id="preprocess_and_train",
        python_callable=preprocess_and_train,
        provide_context=True,
        op_kwargs={"execution_date": "{{ ds }}"}
    )

    # 6. Generar interfaz con Gradio
    gradio_interface_task = PythonOperator(
        task_id="gradio_interface",
        python_callable=gradio_interface,
        provide_context=True,
        op_kwargs={"execution_date": "{{ ds }}"},
    )

    # 7. Finalizar
    end = DummyOperator(task_id="end_pipeline")

    # Definición del flujo de tareas
    start >> create_folders_task >> download_data_task >> split_data_task >> preprocess_and_train_task >> gradio_interface_task >> end
 

![](dag_lineal.png)
![](img.png)
![](img_1.png)

# **2. Paralelizando el Pipeline** (30 puntos)

<p align="center">
  <img src="https://media.tenor.com/vDv7mn58skcAAAAM/clapping.gif" width="300">
</p>



Al ver los resultados obtenidos, Nico queda muy contento con el clasificador. Sin embargo, le aparecen algunas dudas respecto al funcionamiento del pipeline. Primero le comenta que es posible que en un futuro tenga nuevos datos que podrían ser útiles para realizar nuevos entrenamientos, por lo que sería ideal si este pipeline se fuera ejecutando de forma periódica y no de forma manual. Además, Nico le menciona que le gustaría explorar el desempeño de otros modelos además de `Random Forest`, de forma que el pipeline seleccione de forma automática el modelo con mejor desempeño para luego hacer la predicción de Nico.

## **2.1 Preparando un Nuevo Pipeline** (15 puntos)

<p align="center">
  <img src="https://i.makeagif.com/media/7-07-2015/oH6WRw.gif" width="400">
</p>

De acuerdo a lo que le comentó Nico, usted decide crear un nuevo script con las funciones que utilizará su pipeline. Por ende, dentro de la carpeta `dags`, usted creará el archivo `hiring_dynamic_functions.py` el cual debe contener:

1. (2 puntos) Una función llamada `create_folders()` que cree una carpeta, la cual utilice la fecha de ejecución como nombre. Adicionalmente, dentro de esta carpeta debe crear las siguientes subcarpetas:
  - raw
  - preprocessed
  - splits
  - models
2. (2 puntos) Una función llamada `load_ands_merge()` que lea desde la carpeta `raw` los archivos `data_1.csv`y `data_2.csv` en caso de estar disponible. Luego concatene estos y genere un nuevo archivo resultante, guardándolo en la carpeta `preprocessed`.

3. (2 puntos) Una función llamada `split_data()` que lea la data guardada en la carpeta `preprocessed` y realice un hold out sobre esta data. Esta función debe crear un conjunto de entrenamiento y uno de prueba. Mantenga una semilla y 20% para el conjunto de prueba. Guarde los conjuntos resultantes en la carpeta `splits`.

4. (6 puntos) Una función llamada `train_model()` que reciba un modelo de clasificación.
    - La función debe comenzar leyendo el conjunto de entrenamiento desde la carpeta `spits`.
    - Esta debe crear y aplicar un `Pipeline` con una etapa de preprocesamiento. Utilice `ColumnTransformers` para aplicar las transformaciones que estime convenientes.
    - Añada una etapa de entrenamiento utilizando un modelo que ingrese a la función.
  
  Esta función **debe crear un archivo joblib con el pipeline entrenado**. Guarde el modelo con un nombre que le permita una facil identificación dentro de la carpeta `models`.

5. (3 puntos) Una función llamada `evaluate_models()` que reciba sus modelos entrenados desde la carpeta `models`, evalúe su desempeño mediante `accuracy` en el conjunto de prueba y seleccione el mejor modelo obtenido. Luego guarde el mejor modelo como archivo `.joblib`. Su función debe imprimir el nombre del modelo seleccionado y el accuracy obtenido.

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import joblib
from glob import glob

# 1. Función para crear carpetas
def create_folders(execution_date):
    base_folder = os.path.join(os.getcwd(), f"output_{execution_date}")
    subfolders = ["raw", "splits", "models","preprocessed"]
    os.makedirs(base_folder, exist_ok=True)
    for subfolder in subfolders:
        os.makedirs(os.path.join(base_folder, subfolder), exist_ok=True)
        print(f"Creating folders for {os.path.join(base_folder, subfolder)}")

# 2. Leer y concatenar archivos
def load_and_merge(execution_date):
    """
    Lee los archivos `data_1.csv` y `data_2.csv` desde la carpeta `raw`,
    los concatena si ambos existen, y guarda el resultado en `preprocessed`.
    """
    # Definir rutas de las carpetas
    raw_folder = f"output_{execution_date}/raw"
    preprocessed_folder = f"output_{execution_date}/preprocessed"


    # Definir los archivos a buscar
    data_files = [os.path.join(raw_folder, f"data_{i}.csv") for i in range(1, 3)]

    # Leer los archivos que existen
    data_frames = []
    for file in data_files:
        if os.path.exists(file):
            print(f"Leyendo archivo: {file}")
            data_frames.append(pd.read_csv(file))
        else:
            print(f"Archivo no encontrado: {file}")

    if not data_frames:
        raise FileNotFoundError("No se encontraron archivos `data_1.csv` o `data_2.csv` en la carpeta `raw`.")

    # Concatenar DataFrames
    combined_data = pd.concat(data_frames, ignore_index=True)

    # Guardar los datos combinados
    output_file = os.path.join(preprocessed_folder, "combined_data.csv")
    combined_data.to_csv(output_file, index=False)
    print(f"Datos combinados guardados en: {output_file}")

# 3. Dividir datos
def split_data(execution_date):
    """
    Divide el archivo `combined_data.csv` en conjuntos de entrenamiento y prueba (80%-20%).
    Guarda los conjuntos en la carpeta `splits`.
    """
    preprocessed_folder = f"output_{execution_date}/preprocessed"
    splits_folder = f"output_{execution_date}/splits"

    # Leer datos preprocesados
    data_path = os.path.join(preprocessed_folder, "combined_data.csv")
    data = pd.read_csv(data_path)

    # Dividir datos
    X = data.drop(columns=["HiringDecision"])
    y = data["HiringDecision"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # Guardar datos
    pd.concat([X_train, y_train], axis=1).to_csv(os.path.join(splits_folder, "train.csv"), index=False)
    pd.concat([X_test, y_test], axis=1).to_csv(os.path.join(splits_folder, "test.csv"), index=False)

# 4. Entrenar modelo
def train_model(model, execution_date):
    """
    Entrena un modelo de clasificación utilizando los datos de entrenamiento.
    Guarda el modelo entrenado en la carpeta `models`.
    """
    splits_folder = f"output_{execution_date}/splits"
    models_folder = f"output_{execution_date}/models"

    # Leer conjunto de entrenamiento
    train_path = os.path.join(splits_folder, "train.csv")
    train_data = pd.read_csv(train_path)
    X_train = train_data.drop(columns=["HiringDecision"])
    y_train = train_data["HiringDecision"]

    # Configurar preprocesamiento
    numeric_features = ["Age", "ExperienceYears", "DistanceFromCompany", "InterviewScore", "SkillScore", "PersonalityScore"]
    categorical_features = ["Gender", "EducationLevel", "PreviousCompanies", "RecruitmentStrategy"]

    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
    categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
    )

    # Crear pipeline
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("classifier", model),
    ])

    # Entrenar modelo
    pipeline.fit(X_train, y_train)

    # Guardar modelo entrenado
    model_name = type(model).__name__
    joblib.dump(pipeline, os.path.join(models_folder, f"{model_name}.joblib"))

# 5. Evaluar modelos
def evaluate_models(execution_date):
    """
    Evalúa los modelos entrenados y selecciona el mejor basado en `accuracy`.
    Guarda el mejor modelo como `best_model.joblib` en la carpeta `models`.
    """
    splits_folder = f"output_{execution_date}/splits"
    models_folder = f"output_{execution_date}/models"

    # Leer conjunto de prueba
    test_path = os.path.join(splits_folder, "test.csv")
    test_data = pd.read_csv(test_path)
    X_test = test_data.drop(columns=["HiringDecision"])
    y_test = test_data["HiringDecision"]

    # Evaluar modelos
    model_files = glob(os.path.join(models_folder, "*.joblib"))
    if not model_files:
        raise FileNotFoundError("No se encontraron modelos en la carpeta `models`.")

    best_model = None
    best_accuracy = 0
    best_model_name = ""

    for model_file in model_files:
        model = joblib.load(model_file)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        model_name = os.path.basename(model_file).replace(".joblib", "")
        print(f"Modelo: {model_name}, Accuracy: {acc:.4f}")

        if acc > best_accuracy:
            best_accuracy = acc
            best_model = model
            best_model_name = model_name

    # Guardar mejor modelo
    if best_model:
        joblib.dump(best_model, os.path.join(models_folder, "best_model.joblib"))
        print(f"Mejor modelo: {best_model_name}, Accuracy: {best_accuracy:.4f}")
    else:
        raise ValueError("No se pudo determinar el mejor modelo.")

## **2.2 Componiendo un nuevo DAG** (15 puntos)

<p align="center">
  <img src="https://y.yarn.co/977552ab-0b55-4118-9948-06f6386474da_text.gif" width="400">
</p>

Con las nuevas funciones, vamos a crear nuestro nuevo DAG. Para ello, cree un nuevo script en la carpeta `dags`, llamandolo `dag_dynamic.py`. Este script debe contener la siguiente estructura:

1. (1 punto) Inicialice un DAG con fecha de inicio el 1 de octubre de 2024, el cual se debe ejecutar el día 5 de cada mes a las 15:00 UTC. Utilice un `dag_id` interpretable para identificar fácilmente. **Habilite el backfill** para que pueda ejecutar tareas programadas desde fechas pasadas.
2. (1 punto) Comience con un marcador de posición que indique el inicio del pipeline.
3. (2 puntos) Cree una carpeta correspondiente a la ejecución del pipeline y cree las subcarpetas `raw`, `preprocessed`, `splits` y `models` mediante la función `create_folders()`.
4. (2 puntos) Implemente un `Branching`que siga la siguiente lógica:
  - Fechas previas al 1 de noviembre de 2024: Se descarga solo `data_1.csv`
  - Desde el 1 de noviembre del 2024: descarga `data_1.csv` y `data_2.csv`.
  En el siguiente [enlace](https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_2.csv) puede descargar `data_2.csv`.
5. (1 punto) Cree una tarea que concatene los datasets disponibles mediante la función `load_and_merge()`. Configure un `Trigger` para que la tarea se ejecute si encuentra disponible **como mínimo** uno de los archivos.
6. (1 punto) Aplique el hold out al dataset mediante la función `split_data()`, obteniendo un conjunto de entrenamiento y uno de prueba.
7. (2 puntos) Realice 3 entrenamientos en paralelo:
  - Un modelo Random Forest.
  - 2 modelos a elección.
  Asegúrese de guardar sus modelos entrenados con nombres distintivos. Utilice su función `train_model()` para ello.
8. (2 puntos)Mediante la función `evaluate_models()`, evalúe los modelos entrenados, registrando el accuracy de cada modelo en el set de prueba. Luego debe imprimir el mejor modelo seleccionado y su respectiva métrica. Configure un `Trigger` para que la tarea se ejecute solamente si los 3 modelos fueron entrenados y guardados.

`Hint:` Recuerde que puede entregar `kwargs` a sus funciones, como por ejemplo la fecha de ejecución `ds`.

Una vez creado el script, vuelva a construir el contenedor en Docker, acceda a la aplicación web de Airflow, ejecute su pipeline y muestre sus resultados. Adjunte imágenes que ayuden a mostrar el proceso y sus resultados.

Adicionalmente, responda (1 c/u):

- ¿Cual es el accuracy de cada modelo en la ejecución de octubre? ¿Se obtienen los mismos resultados a partir de Noviembre?
- Analice como afectó el añadir datos a sus modelos mediante el desempeño del modelo y en costo computacional.
- Muestre el esquema de su DAG ejecutado en octubre y en noviembre.


`Nota:` Para esta sección no debe implementar la tarea en gradio, solamente se espera determinar el mejor modelo y comparar el desempeño obtenido.

**IMPORTANTE: Para esta sección, debe adjuntar todos los scripts creados junto a su notebook en la entrega, ya que serán ejecutados para validar el funcionamiento. Para justificar sus respuestas, adicionaslmente puede utilizar imágenes de apoyo, como screenshots.**

In [ ]:
from airflow import DAG
from airflow.operators.dummy import DummyOperator
from airflow.operators.python import PythonOperator, BranchPythonOperator
from airflow.operators.bash import BashOperator
from airflow.utils.trigger_rule import TriggerRule
from datetime import datetime
from hiring_dynamic_functions import create_folders, load_and_merge, split_data, train_model, evaluate_models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import os
from datetime import datetime, timezone


from datetime import datetime, timezone

def decide_which_datasets(**kwargs):
    """
    Decide qué datasets descargar basado en la fecha de ejecución.
    """
    # Obtener execution_date del contexto
    execution_date = kwargs['ds']

    # Crear fecha límite
    cutoff_date = '2024-11-01'

    # Validar y comparar las fechas
    if execution_date < cutoff_date:
        print(f"Execution date {execution_date} es anterior a {cutoff_date}. Descargando data_1.csv.")
        return "download_data_1"
    else:
        print(f"Execution date {execution_date} es igual o posterior a {cutoff_date}. Descargando data_1.csv y data_2.csv.")
        return "download_data_1_and_2"


# Configuración del DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
}

with DAG(
    dag_id="hiring_dynamic_pipeline",
    default_args=default_args,
    description="Pipeline dinámico para entrenamiento de modelos de contratación",
    schedule_interval="0 15 5 * *",  # Ejecutar el día 5 de cada mes a las 15:00 UTC
    start_date=datetime(2024, 10, 1),
    catchup=True,  # Habilitar backfill
) as dag:

    # Marcador de inicio
    start = DummyOperator(task_id="start_pipeline")

    # Crear carpetas
    create_folders_task = PythonOperator(
        task_id="create_folders",
        python_callable=create_folders,
        op_kwargs={"execution_date": "{{ ds }}"},
    )

    branch_task = BranchPythonOperator(
        task_id="decide_datasets",
        python_callable=decide_which_datasets,
        provide_context=True,  # Para acceder a execution_date
    )

    # Descargar data_1.csv
    download_data_1_task = BashOperator(
        task_id='download_data_1',
        bash_command=(
            "curl -o /root/airflow/output_{{ ds }}/raw/data_1.csv "
            "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv"
        ),
    )

    # Descargar data_1.csv y data_2.csv
    download_data_1_and_2_task = BashOperator(
        task_id='download_data_1_and_2',
        bash_command=(
             "curl -o /root/airflow/output_{{ ds }}/raw/data_1.csv "
        "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv && "
        "curl -o /root/airflow/output_{{ ds }}/raw/data_2.csv "
        "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_2.csv"
    ),
)

    # Concatenar datasets
    load_and_merge_task = PythonOperator(
        task_id="load_and_merge",
        python_callable=load_and_merge,
        op_kwargs={"execution_date": "{{ ds }}"},
        trigger_rule=TriggerRule.ONE_SUCCESS,  # Ejecutar si al menos una rama del Branch fue exitosa
    )

    # Aplicar hold-out
    split_data_task = PythonOperator(
        task_id="split_data",
        python_callable=split_data,
        op_kwargs={"execution_date": "{{ ds }}"},
    )

    # Entrenamiento de modelos
    train_rf_task = PythonOperator(
        task_id="train_random_forest",
        python_callable=train_model,
        op_kwargs={"model": RandomForestClassifier(random_state=42), "execution_date": "{{ ds }}"},
    )
    train_logreg_task = PythonOperator(
        task_id="train_logistic_regression",
        python_callable=train_model,
        op_kwargs={"model": LogisticRegression(max_iter=1000), "execution_date": "{{ ds }}"},
    )
    train_svc_task = PythonOperator(
        task_id="train_svc",
        python_callable=train_model,
        op_kwargs={"model": SVC(), "execution_date": "{{ ds }}"},
    )

    # Evaluar modelos
    evaluate_models_task = PythonOperator(
        task_id="evaluate_models",
        python_callable=evaluate_models,
        op_kwargs={"execution_date": "{{ ds }}"},
        trigger_rule=TriggerRule.ALL_SUCCESS,  # Ejecutar solo si todos los modelos fueron entrenados
    )

    # Marcador de fin
    end = DummyOperator(task_id="end_pipeline")

    # Flujo de tareas
    start >> create_folders_task >> branch_task
    branch_task >> [download_data_1_task, download_data_1_and_2_task]
    [download_data_1_task, download_data_1_and_2_task] >> load_and_merge_task >> split_data_task
    split_data_task >> [train_rf_task, train_logreg_task, train_svc_task] >> evaluate_models_task >> end

### Metricas octubre
![](metricas_octubree.png)

### Metricas a noviembre
![](metricas_noviembre.png)

>Durante noviembre, las métricas muestran una tendencia general al alza, con la excepción de SVC, que mantiene un comportamiento distinto.
>
>En cuanto a los tiempos de ejecución, estos fueron muy similares, con diferencias menores a 5 segundos entre ellos. Sin embargo, SVC destaca por un incremento en el tiempo de entrenamiento en comparación con octubre, así como un aumento en el tiempo de evaluación. Ambos incrementos están directamente relacionados con la mayor cantidad de datos procesados durante este período.





### Ejecución de octubre
![Ejecucion de octubre](dag_octubre.png)
### Ejecucion de Noviembre
![Ejecucion de noviembre](dag_noviembre.png)

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

<center>
<img src ="https://media0.giphy.com/media/W12WAzuqod9VS/200w.gif?cid=6c09b952gekq3fm1no1ttwcvgm9oj3khbm6yxbe6qwnx3nad&ep=v1_gifs_search&rid=200w.gif&ct=g" width = 400 />
